# Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import MinMaxScaler 
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings('ignore')

# Data

In [ ]:
boston_data = datasets.load_boston()
data = pd.DataFrame(boston_data['data'], columns=boston_data['feature_names'])
target = pd.DataFrame(boston_data['target'], columns=['Target'])
df = pd.concat([data, target], axis=1)

In [ ]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# Split data

In [ ]:
X = df.drop(['Target'],axis=1)
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state = 100)

In [ ]:
print('X_train:',X_train.shape, 'y_train:',y_train.shape)
print('X_test:',X_test.shape, 'y_test:',y_test.shape)

X_train: (404, 13) y_train: (404,)
X_test: (102, 13) y_test: (102,)


# DNN + GridSearchCV


In [ ]:
def DNN(units_list, learning_rate):
  model = Sequential()

  for units in units_list:
      model.add(Dense(units = units, kernel_initializer = 'uniform', activation = 'relu'))
      
  model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'relu'))
  model.compile(optimizer =  Adam(learning_rate = learning_rate), loss = 'mse')

  return model

model = KerasRegressor(build_fn=DNN, verbose=0)

pipe = Pipeline([
      ("scaler", MinMaxScaler()),
      ("model", model)
  ])

param_grid = {
      "model__units_list" : [[13,26,13], [26,13,7]],
      "model__epochs": [50, 100, 150],
      "model__batch_size": [8, 16, 32],
      "model__learning_rate": [0.1, 0.01, 0.001]
  }

grid_dnn = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs= -1, verbose=0)

grid_dnn.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('model',
                                        <keras.wrappers.scikit_learn.KerasRegressor object at 0x7f762e21dc50>)]),
             n_jobs=-1,
             param_grid={'model__batch_size': [8, 16, 32],
                         'model__epochs': [50, 100, 150],
                         'model__learning_rate': [0.1, 0.01, 0.001],
                         'model__units_list': [[13, 26, 13], [26, 13, 7]]})

In [ ]:
result_df = pd.DataFrame(grid_dnn.cv_results_)

In [ ]:
result_df[['params', 'mean_test_score', 'rank_test_score']].head()

,params,mean_test_score,rank_test_score
0,"{'model__batch_size': 8, 'model__epochs': 50, ...",-24.810915,12
1,"{'model__batch_size': 8, 'model__epochs': 50, ...",-129.682122,21
2,"{'model__batch_size': 8, 'model__epochs': 50, ...",-134.027596,30
3,"{'model__batch_size': 8, 'model__epochs': 50, ...",-130.033768,22
4,"{'model__batch_size': 8, 'model__epochs': 50, ...",-23.982730,11


In [ ]:
print('best params:', grid_dnn.best_params_)
print('best score:', grid_dnn.best_score_)

best params: {'model__batch_size': 16, 'model__epochs': 100, 'model__learning_rate': 0.1, 'model__units_list': [26, 13, 7]}
best score: -14.552474021911621


# Prediction

In [ ]:
pred = grid_dnn.predict(X_test)

#RMSE 
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : {}".format(rmse))

RMSE : 24.263029650955325
